In [6]:
import numpy as np
import pandas as pd

**Conjuntos** del modelo de optimización

| **Término** | **Descripción**                |
|-------------|--------------------------------|
| I           | Conjunto de productores        |
| J           | Conjunto de centros de acopio  |
| K           | Conjunto de clientes           |
| P           | Conjunto de productos          |
| T           | Conjunto de periodos de tiempo |

In [18]:
# Variables
# I = 'Productores'
J = 'CAcopios'
K = 'Clientes'
P = 'Productos'
# T = 'Tiempo'

In [10]:
csv_base = './csv/'

routes = {
    # Costo fijo del centro de acopio
    'cfa': 'CostoFijoCA.csv',
    # Costo variable del centro de acopio
    'cva' : 'CostoVarCA.csv',
    # Costo de inventario del centro de acopio
    'cia' : 'CostoInvCA.csv',
    # Costo de transporte del centro de acopio al cliente
    'ctac' : 'CostoTransCAClie.csv'
}

csv_config = {
    'delimiter' : ';',
    'decimal' : ','
}

# pd.read_csv(r'' + csv_base + routes.get('...'), delimiter=csv_config.get('delimiter'), decimal=csv_config.get('decimal'))

In [15]:
J_df = pd.read_csv(csv_base + J + '.csv')
J_df

,CAcopios
0,San Juan del Cesar
1,Riohacha
2,Maicao
3,Guamal
4,Pivijay
5,Nueva Granada
6,Monteria
7,Chinu
8,Planeta rica


In [19]:
K_df = pd.read_csv(csv_base + K + '.csv')
K_df

,Clientes
0,Distribuidor1
1,Distribuidor2
2,Distribuidor3
3,Tienda1
4,Tienda2
5,Mercado1
6,Mercado2
7,Mercado3
8,Panaderia1
9,Panaderia2


In [12]:
P_df = pd.read_csv(csv_base + P + '.csv')
P_df.head()

,Productos
0,Queso duro
1,Queso blando


## Escenarios de prueba de demandas por centro de acopio
Escenario 1: Un centro de acopio sobrepasa la demanda del cliente
- El modelo puede asignar dicho centro de acopio para suplir con la demanda
- El centro de acopio debe proveer esa demanda

Escenario 2: Ningún centro de acopio cumple con toda la demanda del cliente
- El modelo debe dar una solución de varios centros de acopio
- La suma de la demanda por cada centro de acopio debe ser igual a la demanda

Escenario 3: Un centro de acopio tiene en stock exactamente la cantidad de la demanda de un cliente.
- El modelo puede tomar el centro de acopio que tiene en stock exactamente la demanda a cumplir

### Generando demandas por cliente

In [ ]:
import random

K_df = pd.read_csv(r'' + csv_base + K + '.csv')
clients = K_df[K].drop_duplicates()

X = []

for client in clients:
  X.append(10 * random.randint(0, 100))

clients = pd.DataFrame(clients)

clients['Demanda'] = pd.DataFrame(X)
clients

In [ ]:
# Tomando una demanda
clients.loc[0]

### Generando capacidades para cada centro de acopio

In [ ]:
J_df = pd.read_csv(r'' + csv_base + J + '.csv')
c_acopios = J_df['Cacopio']
X = []

for c_acopio in c_acopios:
  X.append(10 * random.randint(0, 100))

c_acopios = pd.DataFrame(c_acopios)
c_acopios['Capacidad'] = pd.DataFrame(X)
c_acopios

### func. obj

$$
Min(F) = CProduccion_t + COperacion_t + CInventario_t + CTransporte_t
$$

#### Costos de producción
$$
  CProduccion_t = \sum_{i \in I} \sum_{j \in J} \sum_{p \in P} CostoProduccion_{pit} PA_{pijt} \qquad \forall_t \in T
$$

In [ ]:
cp_df = pd.read_csv(r'' + csv_base + routes.get('c_produccion'))

cp_sum = cp_df.groupby('Productores')['CostoProduccion'].agg('sum')
cp_sum

#### Costos de Operación
$$
  COperacion_t = CFijos_t + CVariables_t
$$

In [ ]:
cfa_df = pd.read_csv(r'' + csv_base + routes.get('c_fijo_acopio'))

cfa_sum = cfa_df.groupby('CAcopios')['CostoFijoAcopio'].agg('sum')
cfa_sum

In [ ]:
cva_df = pd.read_csv(r'' + csv_base + routes.get('c_var_acopio'))
cva_sum = cva_df.groupby('CAcopios')['CostoVarAcopio'].agg('sum')
cva_sum

#### Costos de Inventario
$$
  CInventario_t = \sum_{j \in J} \sum_{p \in P} CostoInvAcopio_{pjt} InvCA_{pjt} \qquad \forall_t \in T
$$

In [ ]:
cia_df = pd.read_csv(r'' + csv_base + routes.get('c_inv_acopio'))

cia_sum = cia_df.groupby('CAcopios')['CostoInvAcopio'].agg('sum')
cia_sum

#### Costos de Transporte
$$
  CTransporte_t = \sum_{j \in J} \sum_{i \in I} \sum_{p \in P} CostoTransAcopio_{pijt} PA_{pijt} + \sum_{k \in K} \sum_{j \in J} \sum_{p \in P} \qquad \forall_t \in T
$$

In [ ]:
ctpa_df = pd.read_csv(r'' + csv_base + routes.get('c_trans_prod_acopio'))

ctpa_sum = ctpa_df.groupby('CAcopios')['CostoTransProdAcopio'].agg('sum')
ctpa_sum

In [ ]:
ctac_df = pd.read_csv(r'' + csv_base + routes.get('c_trans_acopio_clie'))

ctac_sum = ctac_df.groupby('CAcopios')['CostoTransAcopioClie'].agg('sum')
ctac_sum

### scipy

In [ ]:
from scipy.optimize import dual_annealing

def f(x):
  x = x * sum(clients['Demanda'].values.tolist())
  return x

bounds = [(-2000, 2000)]
result = dual_annealing(f, bounds=bounds)
print(result)

In [ ]:
dc_df = pd.read_csv(r'' + csv_base + routes.get('demanda_clie'))
res = dc_df.groupby('Clientes')['DemandaClie'].agg('sum')

# Capacidades

In [ ]:
cpr_df = pd.read_csv(r'' + csv_base + routes.get('cap_productor'))
cpr_df.head()

In [ ]:
cac_df = pd.read_csv(r'' + csv_base + routes.get('cap_almacenamiento_ca'))
cac_df.head()